# Edición de DataFrames

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data/IGC.annotation.tsv.gz', sep='\t')

## Descripción de dataframe

¿Recuerdas algunos métodos para explorar un DataFrame?

## Ausencia de valores 

El método `isnull()` detecta ausencia de valores, devuelve un valor booleano, `True/False`.

In [ ]:
df.isnull()

`NaN`, es para *not a number*, es un tipo de dato **numérico** usado para representar cualquier valor que no está definido o representado en nuestra tabla. Para Pandas ambos métodos, `isna()` == `isnull()`, [son idénticos](https://github.com/pandas-dev/pandas/blob/537b65cb0fd2aa318e089c5e38f09e81d1a3fe35/pandas/core/dtypes/missing.py#L109). Si queremos usar NaN como valor numérico, debemos de usar `np.isnan()` de Numpy.

El encadenamiento de métodos nos permite obtener la suma de los valores `True` de cada una de las Series.

In [ ]:
df.isnull().sum()

La suma de todos los valores `True` de todo el DataFrame.

In [ ]:
df.isnull().sum().sum()

El método `any()` devuelve un valor booleano si presente aunque sea un solo elemento NaN.

In [ ]:
df.isnull().any()

Si el atributo `columns` nos da las columnas, podemos usar el resultado de la cadena de métodos anterior para obtener únicamente las columnas `True`.

In [ ]:
df.columns

In [ ]:
df.columns[df.isnull().any()]

La selección de algún método o de índices de un DataFrame siempre seleccionará los valores verdaderos, **`df[True]`**, entonces podemos usar el resultado anterior, cuyo *datatype* es Index, y usarlo para seleccionar columnas de un DataFrame. A la siguiente instrucción se le colocó un poco más de espacio para dejar en claro el concepto.

In [ ]:
df[df.columns[df.isnull().any()]]

Se puede una manera más estructurada:

In [ ]:
tienen_nulls = df.isnull().any()
columnas_que_tienen_nulls = df.columns[tienen_nulls]
dataframe_con_columnas_que_contiene_nulls = df[columnas_que_tienen_nulls]

In [ ]:
dataframe_con_columnas_que_contiene_nulls

La sintáxis anterior contiene variables con nombres muy informativos, esto es útil cuando se utiliza python en modo *scripting*. Una de las ventajas que ofrece Jupyter es documentar código en un texto enriquecido en formato Markdown en celdas independientes al código, ideal para reportes. 

Podemos usar la virgulilla -*"la tilde de la eñe"*- para indicar *negación*, lo puesto a la expresión evaluada por `isnull().any()`, es decir los valores `False`, aquellas columnas que contienen valores `NaN`.

In [ ]:
df.columns[~df.isnull().any()]

In [ ]:
df[df.columns[~df.isnull().any()]]

Lo anterior también se puede aplicar a filas, con la opción `axis=1`.

In [ ]:
df.isnull().any(axis=1)

In [ ]:
df[df.isnull().any(axis=1)]

In [ ]:
df[~df.isnull().any(axis=1)]

In [ ]:
nulls_en_filas = df.isnull().any(axis=1)
df_con_null_en_filas = df[nulls_en_filas]
df_con_null_en_filas

In [ ]:
nulls_en_filas = df.isnull().any(axis=1)
df_SIN_null_en_filas = df[~nulls_en_filas]
df_SIN_null_en_filas

¿Qué significa la siguiente línea?

In [ ]:
len(df[~df.isnull().any(axis=1)].iloc[0:4, 1:4].describe().T.shape)

In [ ]:
df.fillna(0).head()

Veamos las primeras 5 filas que presentan al menos 1 valor `NaN`.

In [ ]:
df[df.isnull().any(axis=1)].head()

Y ahora llenemos esos valores con cero.

In [ ]:
df[df.isnull().any(axis=1)].fillna(0).head()

## Limpieza de *DataFrames*

### Columnas

El método upper funciona con *strings*

In [ ]:
df.columns.upper()

In [ ]:
df.columns.str.upper()

In [ ]:
df.columns.str.lower()

In [ ]:
df.columns.str.title()

Podemos usar `replace()`, un método que funciona con strings, para poder buscar y reemplazar caracteres que deseemos eliminar.

In [ ]:
df.columns.str.lower().str.replace('(', ' ', regex=True)

In [ ]:
df.columns.str.lower().str.replace('(', ' ', regex=False).str.replace(')', '', regex=False)

In [ ]:
df.columns.str.lower().str.replace('(', ' ', regex=False).str.replace(')', '', regex=False).str.replace(' ', '_', regex=False)

In [ ]:
df.columns.str.lower().str.replace('(', ' ', regex=False).str.replace(')', '', regex=False).str.replace(' ', '_', regex=False).str.strip()

El método `strip()` elimina los espacios a la izquierda y derecha:
* `strip()` = `lstrip()` + `rstrip()`

Los paréntesis nos permiten escribir encadenamiento de métodos largos sin necesidad de usar break-lines.
```python
  (
    df.columns.str.lower()
     .str.replace('(', ' ', regex=False)
     .str.replace(')', '', regex=False)
     .str.replace(' ', '_', regex=False).str.strip()
  )
```

In [ ]:
df.columns = (df.columns.str.lower()
                .str.replace('(', ' ', regex=False)
                .str.replace(')', '', regex=False)
                .str.replace(' ', '_', regex=False).str.strip())

In [ ]:
df.columns

Limpieza en el contenido de cada columna

### Valores

In [ ]:
df['gene_name']

Con el método `unique()` podemos obtener los valores únicos de cada columna.

In [ ]:
df['gene_completeness'].unique()

Ahora podemos ordenar los valores únicos, algo similar a `sort | uniq` en Linux.

In [ ]:
sorted(df['gene_completeness'].unique())

Python nos es cohersivo con los tipos de datos por columnas, `gene_completeness` es tipo `Object`, sin embargo, los tipos de datos de `unique()` no son todos del mismo tipo, hay valores `nan`.

In [ ]:
sorted(df['gene_completeness'].unique().astype(str))

Seleccionemos otras columnas pra contar u ordenar.

In [ ]:
df.columns

In [ ]:
sorted(df['taxonomic_annotation_phylum_level'].unique())

In [ ]:
sorted(df['taxonomic_annotation_genus_level'].unique())

Algunas columnas pueden contener caracteres que podrían dificultar el parseo posterior

In [ ]:
df['cohort_assembled'].unique()

Hacemos la sustitución y vemos si se realiza el cambio con `head()`

In [ ]:
df['cohort_assembled'].str.replace(';','_').head(10)

Se guardan los cambios definiendo la columna con los cambios realizados por el reemplazo.

In [ ]:
df['cohort_assembled'] = df['cohort_assembled'].str.replace(';','_')

In [ ]:
df['cohort_assembled'].unique()

In [ ]:
df['cohort_origin'].unique()

¿Como podemos contar cuántos elementos únicos hay?

In [ ]:
df.groupby('cohort_origin')['cohort_origin'].count()

El agrupamiendo de columnas tiene varios usos:

* Divide la tabla en grupos
* Aplica operaciones a esas tablas pequeñas
* Combina los resultados

In [ ]:
df.groupby('cohort_origin')

In [ ]:
df.groupby('cohort_origin').mean()

Obtengamos la longitud máxima y mínima de los genes, utilizando índices.

In [ ]:
sorted(df['gene_length'].unique())[0]

In [ ]:
sorted(df['gene_length'].unique())[-1]

Obtengamos la longitud máxima y mínima de los genes, utilizando los métodos `min()` y `max()`.

In [ ]:
df['gene_length'].min()

In [ ]:
df['gene_length'].max()

### Eliminación de columnas

In [ ]:
df.head()

In [ ]:
# axis 0 para filas y axis 1 para columnas
df = df.drop('gene_completeness', axis=1)

In [ ]:
# df.drop('gene_completeness', axis=1, inplace=True)

In [ ]:
# df = df.drop(['cohort_assembled', 'gene_completeness'], axis=1)

In [ ]:
df.columns

## Filtrado de columnas

In [ ]:
df.columns

In [ ]:
df['cohort_origin'].head()

In [ ]:
df['cohort_origin'] == 'EUR'

In [ ]:
df[df['cohort_origin'] == 'EUR']

In [ ]:
df[df['taxonomic_annotation_phylum_level'] == 'Firmicutes']

In [ ]:
df[df["taxonomic_annotation_genus_level"] == 'Salmonella']

In [ ]:
df[df['taxonomic_annotation_genus_level'].str.contains('Salmonella')]

In [ ]:
df[df['taxonomic_annotation_genus_level'].str.contains('sAlmonELLa', case=False)]

In [ ]:
df[df['taxonomic_annotation_genus_level'].str.contains('Salmonella')].shape

In [ ]:
df[df['taxonomic_annotation_genus_level'].str.contains('Escherichia')].shape

In [ ]:
df[df['taxonomic_annotation_genus_level'].str.contains('|'.join(['Salmonella', 'Escherichia']))]

In [ ]:
df[df['taxonomic_annotation_genus_level'].str.contains('|'.join(['Salmonella', 'Escherichia']))].shape

## Recapitulando

In [ ]:
df = (df[(df['cohort_origin'] == 'EUR') & 
         (df['kegg_annotation'] != 'unknown') &
         (df['eggnog_annotation'] != 'unknown') &
         (df['taxonomic_annotation_phylum_level'] != 'unknown') &
         (df['taxonomic_annotation_genus_level'] != 'unknown')]).reset_index(drop=True)

```python
#df_conditionA: df[column] == something

df = ( df[ (df_conditionA) &
           (df_conditionB) &
           (df_conditionC) &
           (df_conditionD) ] )
```

In [ ]:
df = df.drop('cohort_assembled', axis=1)

In [ ]:
df.head()

In [ ]:
df.to_csv('data/igc_prefiltered.tsv.gz', sep='\t', index=False)

## Gráficas con Pandas

In [ ]:
df.head()

In [ ]:
df['gene_length'].plot.line();

In [ ]:
df['gene_length'].plot.hist();

In [ ]:
df['taxonomic_annotation_phylum_level'].unique()

In [ ]:
df['cohort_origin'].value_counts().plot.bar();

In [ ]:
df['taxonomic_annotation_phylum_level'].value_counts().plot.bar();